### Import

In [14]:
!conda install pytorch-tabnet

Channels:
 - defaults
Platform: win-64
Solving environment: ...working... failed



PackagesNotFoundError: The following packages are not available from current channels:

  - pytorch-tabnet

Current channels:

  - https://repo.anaconda.com/pkgs/main
  - https://repo.anaconda.com/pkgs/r
  - https://repo.anaconda.com/pkgs/msys2

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [12]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import OrdinalEncoder

ImportError: cannot import name 'ArrowDtype' from 'pandas.core.dtypes.dtypes' (c:\Users\vhehr\.conda\envs\CatAndTab\lib\site-packages\pandas\core\dtypes\dtypes.py)

In [9]:
import pyarrow
print(pyarrow.__version__)


11.0.0


### Data Load

In [31]:
train = pd.read_csv('./Data/train.csv').drop(columns=['ID'])
test = pd.read_csv('./Data/test.csv').drop(columns=['ID'])

In [32]:
X = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

### Data Pre-processing

In [33]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이",
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "총 시술 횟수",
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수",
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수",
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",
    "정자 출처",
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [34]:
# 카테고리형 컬럼들을 문자열로 변환
for col in categorical_columns:
    X[col] = X[col].astype(str)
    test[col] = test[col].astype(str)

In [35]:
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

X_train_encoded = X.copy()
X_train_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

X_test_encoded = test.copy()
X_test_encoded[categorical_columns] = ordinal_encoder.transform(test[categorical_columns])

In [9]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",
    "미세주입 배아 이식 수",
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수",
    "수집된 신선 난자 수",
    "저장된 신선 난자 수",
    "혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수",
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일",
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일",
    "배아 해동 경과일"
]

In [36]:
numeric_columns = [col for col in numeric_columns if col in X.columns]

X_train_encoded[numeric_columns] = X_train_encoded[numeric_columns].fillna(0)
X_test_encoded[numeric_columns] = X_test_encoded[numeric_columns].fillna(0)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_encoded, y, test_size=0.2, random_state=42, stratify=y
)

### Train

TabNet

In [ ]:
import optuna
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

# Optuna 최적화 함수
def objective(trial):
    # 최적화할 하이퍼파라미터 정의
    params = {
        "n_d": trial.suggest_int("n_d", 8, 64),
        "n_a": trial.suggest_int("n_a", 8, 64),
        "n_steps": trial.suggest_int("n_steps", 3, 10),
        "gamma": trial.suggest_float("gamma", 1.0, 2.0),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 0.0001, 0.01),
        "momentum": trial.suggest_float("momentum", 0.01, 0.4),
        "epochs": trial.suggest_int("epochs", 50, 300),
        "batch_size": trial.suggest_int("batch_size", 128, 1024),
        "virtual_batch_size": trial.suggest_int("virtual_batch_size", 16, 128)
    }

    # TabNet 모델 생성 및 학습
    model = TabNetClassifier(**params, verbose=0, seed=42)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], patience=10, batch_size=params["batch_size"], virtual_batch_size=params["virtual_batch_size"])

    # 검증 데이터에서 예측 확률 계산
    y_valid_proba = model.predict_proba(X_valid)[:, 1]

    # ROC-AUC 점수 반환 (최적화 대상)
    return roc_auc_score(y_valid, y_valid_proba)




In [ ]:
# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=150, show_progress_bar=True)  # 30번의 실험 진행 (더 늘리면 정확도 향상 가능)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters:", study.best_params)

In [ ]:
import joblib

# Optuna 최적화 실행 후 최적의 study 저장
joblib.dump(study, "TabNet_optuna_study.pkl")


['xgb_optuna_study.pkl']

In [49]:
import joblib

# 저장된 Optuna study 불러오기
study = joblib.load("xgb_optuna_study.pkl")

# 최적의 하이퍼파라미터 가져오기
best_params = study.best_params
print("Best Params:", best_params)

# 최적의 하이퍼파라미터로 모델 학습
model = xgb.XGBClassifier(**best_params, random_state=42, n_jobs=-1)
model.fit(X_train_encoded, y)


FileNotFoundError: [Errno 2] No such file or directory: 'xgb_optuna_study.pkl'

In [69]:
# 최적 하이퍼파라미터 가져오기
best_params = study.best_params

# 최적의 파라미터로 모델 재학습
model = xgb.XGBClassifier(
    **best_params,
    objective="binary:logistic",
    eval_metric="auc",
    use_label_encoder=False,
    verbosity=0,
    random_state=42,
    n_jobs=-1
)

In [70]:
model.fit(X_train_resampled, y_resampled)

XGBClassifier(alpha=0.01734993356678516, base_score=None, booster=None,
              callbacks=None, colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.5125729570251544, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', feature_types=None, gamma=1.5197921362681555,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, lambda=1.8940631696982866,
              learning_rate=0.01825270924751515, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=14, max_leaves=None,
              min_child_weight=7, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=750, n_jobs=-1, ...)

In [71]:
from sklearn.metrics import accuracy_score, roc_auc_score

# 예측
y_train_pred = model.predict(X_train_encoded)
y_train_proba = model.predict_proba(X_train_encoded)[:, 1]  # ROC-AUC Score 계산용

# 평가
accuracy = accuracy_score(y, y_train_pred)
roc_auc = roc_auc_score(y, y_train_proba)

# 출력
print(f"Accuracy: {accuracy:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")

Accuracy: 0.7784
ROC-AUC Score: 0.8090


RandomForest (Hyperparameter tune)

In [52]:
model.save_model("xgboost_SMOTE_model.json")

### Predict

In [72]:
pred_proba = model.predict_proba(X_test_encoded)[:, 1]

### Submission

In [73]:
sample_submission = pd.read_csv('./Data/sample_submission.csv')
sample_submission['probability'] = pred_proba

In [75]:
sample_submission.to_csv('./XGBoost_SMOTE3_optimization_submit.csv', index=False)